### Imports

In [1]:
# Imports
import requests
import pandas as pd
import os
import tempfile
import glob

import newspaper
from newspaper import fulltext, Article

from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine

from bs4 import BeautifulSoup
from dotenv import load_dotenv
import html5lib

### Functions

In [3]:
def make_soup(url):
    page = requests.get(url) 
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

### Connect to PostgreSQL

In [2]:
# # Load .env file
# load_dotenv('.env')

# # Get credentials from .env
# config = dict(
#     drivername='postgres',
#     username=os.getenv("USERNAME"),
#     password=os.getenv("PASSWORD"),
#     host=os.getenv("HOST"),
#     database=os.getenv("DATABASE")
# )

# # Create the connection string
# url = URL(**config)
# print(url)

# # Create the engine
# engine = create_engine(url)

### Get news categories

In [4]:
# Make soup
website_url = 'https://foxnews.com'
soup = make_soup(website_url)

In [5]:
# Extract navigation items
nav_items = soup.find_all("li", class_="nav-item")

In [6]:
# Construct empty list
news_categories = []

# Extract category name and url for each category
for item in nav_items:
    output = dict(category = item.text,
                  url = item.find_all('a')[0].get('href'))
    news_categories.append(output)
    
# Place in a dataframe (and drop duplicate categories)
news_categories = pd.DataFrame(news_categories)

# Write to dataframe
news_categories.to_csv('data/news-categories-fox.csv', index=False)

### Get political article URLs

In [7]:
# # Get political news only
# category = 'Politics'

# # Extract url for political news
# category_url = news_categories.loc[news_categories['category'] == category, 'url'].item()

In [8]:
# Manually set category url for politics
category_url = 'https://www.foxnews.com/politics'
soup = make_soup(category_url)

In [9]:
# Make list of all article items
article_items = soup.find_all("article")

In [10]:
# Extract category name and url for each category
articles = []
for item in article_items:
    if 'video.' not in item.find_all('a')[0].get('href'):
        output = dict(headline = item.text,
                      url = website_url + item.find_all('a')[0].get('href'))
        articles.append(output)

In [11]:
# Write to dataframe
pd.DataFrame(articles).to_csv('data/article-urls-fox.csv', index=False)

### Full text articles

In [12]:
# full_texts = []

# for a in articles[0:4]:
#     page = requests.get(a['url'])
#     soup = BeautifulSoup(page.content, 'html.parser')
# #     full_text = soup.find_all('article')[0].text #This line is wrong
#     full_texts.append(full_text)

#### The following works for one specific article

In [177]:
# 'https://foxnews.com/politics/texas-bishop-backs-up-priest-who-said-catholic-democrats-should-repent-or-face-the-fires-of-hell'

In [231]:
soup = make_soup(articles[3]['url'])

In [234]:
# Get all <p>'s that contain sentences from the article
# sentences = soup.article.find_all('div')[7].contents[0].contents[0].find_all('p')
sentences = soup.article.find_all('p')

# Remove <p>'s that are random links to other articles
sentences = [sent for sent in sentences if '<p><a href' not in str(sent)]

# Extract the text from all the <p>'s'
sentences = [sent.text for sent in sentences]

# Join all individual sentences
full_text = ' '.join(sentences)

full_text

'DNI defends scaling back in-person briefings on election security, citing potential for leaks; reaction and analysis on \'Outnumbered.\' The House Intelligence Committee received a whistleblower complaint claiming Trump administration officials at the White House and the Department of Homeland Security suppressed intelligence reports related to Russian election interference, Democratic Committee Chairman Adam Schiff said Wednesday. In a statement, Schiff said former Intelligence and Analysis Acting Under Secretary Brian Murphy filed the whistleblower reprisal complaint on Tuesday with the DHS Office of Inspector General. “We’ve received a whistleblower complaint alleging DHS suppressed intel reports on Russian election interference, altered intel to match false Trump claims and made false statements to Congress,” Schiff, D-Calif., tweeted Wednesday afternoon. “This puts our national security at risk.” He added: “We will investigate.” Schiff said that the investigation will come as par

In [213]:
# But this isn't the title...
title = soup.title.string
title

"Nick Searcy discusses new documentary 'America, America, God Shed His Grace on Thee' | Fox News"